In [ ]:
import forestry
import importlib
importlib.reload(forestry)
from forestry import *
import ipywidgets as widgets
import time

In [ ]:
class CLI(Game):
    def __init__(self):
        self.out = widgets.Output()
        self.command_out = widgets.Output()
        self.text = widgets.Text()
        self.text_memory = []
        self.text_counter = -1
        self.button_left = widgets.Button(
            icon='arrow-left'
        )
        self.button_right = widgets.Button(
            icon='arrow-right'
        )
        self.text.on_submit(self.register_command)
        self.button_left.on_click(self.text_previous)
        self.button_right.on_click(self.text_next)
        self.console = widgets.HBox([self.text, self.button_left, self.button_right])
        display(widgets.VBox([self.out, self.command_out, self.console]))
        
        self.print_buffer = []
        self.print_buffer_size = 0
        
        # self.print(type(self.inv[0]), out=self.command_out, flush=True)
        
        super().__init__()
        
        
    def print(self, *strings, sep=' ', end='\n', flush=False, append=False, out=None):
        if out is None:
            out = self.out
        thing = sep.join(map(str, strings)) + end
        
        if append:
            out.append_stdout(thing)
            return
        
        self.print_buffer.append(thing)
        self.print_buffer_size += len(thing)
        if flush:
            out.outputs = ({'output_type': 'stream',
              'name': 'stdout',
              'text': ''.join(self.print_buffer)},) # very evil hack, it will clear the screen every time you flush (and without flush, you won't even see anything)
            self.print_buffer = []
            self.print_buffer_size = 0
        
    def render(self):
        while True:
            if self.exit_event.is_set():
                self.console.layout.display = 'none'
                self.print(flush=True)
                break
                        
            if len(self.to_render) == 0:
                self.print(flush=True)
                continue
                
            if self.render_event.is_set():
                if self.render_help.is_set():
                    self.print(self.help_text, flush=True)
                else:
                    for thing in self.to_render[:-1]:
                        self.print(thing)
                    self.print(self.to_render[-1], flush=True)
                    self.render_event.clear()
                
    def register_command(self, b):
        if self.exit_event.is_set():
            return
        self.print(self.text.value, out=self.command_out, flush=True)
        
        self.execute_command(self.text.value)
        
        if len(self.text_memory) == 0 or self.text.value != self.text_memory[-1]:
            self.text_memory.append(self.text.value)
        self.text_counter = len(self.text_memory)
        self.text.value = ''

    def text_next(self, b):
        self.text_counter = min(self.text_counter + 1, len(self.text_memory))
        if self.text_counter == len(self.text_memory):
            self.text.value = ''
        else:
            self.text.value = self.text_memory[self.text_counter]
    
    def text_previous(self, b):
        self.text_counter = max(self.text_counter - 1, 0)
        if self.text_counter != 0 or len(self.text_memory) != 0:
            self.text.value = self.text_memory[self.text_counter]

In [ ]:
cli = CLI()